In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

### E. coli core model

In [ ]:
import cameo
from cameo import load_model
from cameo.strain_design.heuristic import GeneKnockoutOptimization, ReactionKnockoutOptimization
from cameo.strain_design.heuristic.objective_functions import biomass_product_coupled_yield, product_yield, number_of_knockouts
from cameo.flux_analysis.simulation import fba
from cameo.parallel import SequentialView
import inspyred

In [ ]:
model = load_model("../tests/data/EcoliCore.xml")

In [ ]:
objective1 = biomass_product_coupled_yield(
    "Biomass_Ecoli_core_N_LPAREN_w_FSLASH_GAM_RPAREN__Nmet2",
    "EX_ac_LPAREN_e_RPAREN_",
    "EX_glc_LPAREN_e_RPAREN_")

objective2 = number_of_knockouts()
objective = [objective1, objective2]

In [ ]:
rko = ReactionKnockoutOptimization(model=model,
                                   simulation_method=fba,
                                   objective_function=objective,
                                   heuristic_method=inspyred.ec.emo.NSGA2,
                                   seed=1234)

In [ ]:
results = rko.run(max_evaluations=3000, pop_size=10, view=SequentialView())

In [ ]:
results

In [ ]:
mutant = model.copy()

In [ ]:
mutant.reactions.PGL.knock_out()
mutant.reactions.SUCDi.knock_out()

In [ ]:
mutant_fluxes = mutant.solve().x_dict

In [ ]:
print mutant_fluxes['Biomass_Ecoli_core_N_LPAREN_w_FSLASH_GAM_RPAREN__Nmet2']
print mutant_fluxes['EX_ac_LPAREN_e_RPAREN_']

### iJO1366 model

In [ ]:
model = load_model("../tests/data/iJO1366.xml")

In [ ]:
of1 = product_yield("EX_ac_LPAREN_e_RPAREN_", "EX_glc_LPAREN_e_RPAREN_")
of2 = number_of_knockouts()
of3 = biomass_product_coupled_yield("Ec_biomass_iJO1366_core_53p95M", "EX_ac_LPAREN_e_RPAREN_", "EX_glc_LPAREN_e_RPAREN_")

In [ ]:
ko = GeneKnockoutOptimization(model=model, objective_function=[of1, of2, of3],
                                  simulation_method=fba, heuristic_method=inspyred.ec.emo.NSGA2)

In [ ]:
ko.run(max_evaluations=5000, n=10, mutation_rate=0.3, popultation_size=100)